In [97]:
import pandas as pd
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline
from accelerate import disk_offload
import torch
import os
import transformers
import numpy as np

In [55]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

c:\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicco\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [91]:
def classify_long_text_with_flan(text, token_limit=512):
    # Define a prompt to ask FLAN if the text is relevant
    prompt_template = (
    "Is there any mention of passivating molecules or techniques aimed at improving perovskite solar cell stability in this research paper? "
    "Classify as 'Relevant' if the text refers to either of these topics, even indirectly. Otherwise, answer 'Not Relevant'.\n\nContent: {}"
)
    
    # Tokenize and split into chunks
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunk_size = token_limit - len(tokenizer(prompt_template.format(""))["input_ids"])  # Adjust for prompt length
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    predictions = []
    for chunk in chunks:
        # Decode chunk and add to prompt
        content = tokenizer.decode(chunk, skip_special_tokens=True)
        prompt = prompt_template.format(content)
        
        # Generate response using FLAN
        inputs = tokenizer(prompt, return_tensors="pt", max_length=token_limit, truncation=True)
        outputs = model.generate(**inputs, max_length=50)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Interpret the response
        if prediction == "Relevant":
            predictions.append(1)
        else:
            predictions.append(0)
    
    # Majority voting
    final_prediction = 1 if predictions.count(1) > 0 else 0
    return final_prediction

In [92]:
df = pd.read_csv('../data/merged_label.csv')

In [93]:
df_good = df[df['label'] == 1]
df_good_sample = df_good.head()

df_good_sample['pred'] = df_good_sample['text'].apply(classify_long_text_with_flan)
df_good_sample

C:\Users\nicco\AppData\Local\Temp\ipykernel_13492\29086025.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_good_sample['pred'] = df_good_sample['text'].apply(classify_long_text_with_flan)


,link,label,text,pred
0,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...,1
1,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...,1
2,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...,0
3,https://doi.org/10.1038%2Fs41586-022-04604-5,1,Stability-limiting heterointerfaces of perovsk...,0
4,https://doi.org/10.1038%2Fs41467-022-30426-0,1,Imaging and quantifying non-radiative losses a...,1


In [94]:
df['pred'] = df['text'].apply(classify_long_text_with_flan)

In [98]:
np.mean(df['pred'] == df['label'])

0.6363636363636364